In [1]:
from PIL import Image
import numpy as np
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
landMaskIm = Image.open('land_mask.png').convert('LA')
realMapIm = Image.open('clean_map.png').convert('LA')
landMaskIm_flip = landMaskIm.transpose(Image.FLIP_LEFT_RIGHT)
realMapIm_flip = realMapIm.transpose(Image.FLIP_LEFT_RIGHT)

#realMapIm.show()

#landMaskIm.size

## Find squares of the land mask that have both water and land

In [3]:
npLMI = np.array(landMaskIm)[:,:,0]
npLMIF = np.array(landMaskIm_flip)[:,:,0]
npCMI = np.array(realMapIm)[:,:,0]
npCMIF = np.array(realMapIm_flip)[:,:,0]
#print(npLMI)
print(0 in npLMI, 255 in npLMI, 120 in npLMI)
print(npLMI.shape)
print(npCMI.shape)

True True False
(1015, 1831)
(1015, 1831)


In [4]:
nPics = 10000
subSquareSize = (52,52) # Must be multiple of 4!
maskSubSquares = np.zeros((nPics, subSquareSize[0], subSquareSize[1]))
realSubSquares = np.zeros((nPics, subSquareSize[0], subSquareSize[1]))
subSquareCoords = []
validSquareIndex = 0
while validSquareIndex < nPics:
    xCoord = np.random.randint(0,npLMI.shape[0]-subSquareSize[0])
    yCoord = np.random.randint(0,npLMI.shape[1]-subSquareSize[1])
    #print(xCoord, yCoord)
    candidateSubSquare = npLMI[xCoord:xCoord+subSquareSize[0],
                               yCoord:yCoord+subSquareSize[1]]
    if 50 < np.mean(candidateSubSquare) < 225:
        #print(candidateSubSquare.shape)
        #print(candidateSubSquare)
        subSquareCoords.append((xCoord, yCoord))
        maskSubSquare =  npLMI[xCoord:xCoord+subSquareSize[0],
                               yCoord:yCoord+subSquareSize[1]]
        maskSubSquares[validSquareIndex] = maskSubSquare
        
        realSubSquare =  npCMI[xCoord:xCoord+subSquareSize[0],
                               yCoord:yCoord+subSquareSize[1]]
        realSubSquares[validSquareIndex] = realSubSquare

        validSquareIndex += 1


## Check results of search

In [ ]:
nPicsToShow = 100
nCols = 10
nRows = 2*(nPicsToShow+1) / (nCols)
for i in range(nPicsToShow):
    xCoord = subSquareCoords[i][0]
    yCoord = subSquareCoords[i][1]
    plt.subplot(nRows, nCols, (i*2)+1)
    #maskSubSquare = npLMI[xCoord:xCoord+subSquareSize[0],
    #                      yCoord:yCoord+subSquareSize[1]]
    plt.imshow(maskSubSquares[i])
    plt.subplot(nRows, nCols, (i*2)+2)
    #realSubSquare = npCMI[xCoord:xCoord+subSquareSize[0],
    #                      yCoord:yCoord+subSquareSize[1]]
    plt.imshow(realSubSquares[i])
#plt.tight_layout()
plt.show()

## Start machine learning

In [5]:
from keras.models import Model, Sequential
from keras.layers import *
from keras.optimizers import Adam
import keras
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU


Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/home/j5wagner/miniconda2/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/j5wagner/miniconda2/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/j5wagner/miniconda2/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
ImportError: /lib64/libc.so.6: version `GLIBC_2.16' not found (required by /home/j5wagner/miniconda2/lib/python2.7/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so)


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/install_sources#common_installation_problems

for some common reasons and solution

In [7]:
X_train = maskSubSquares
Y_train = realSubSquares

# Scaling the range of the image to [-1, 1]
# Because we are using tanh as the activation function in the last layer of the generator
# and tanh restricts the weights in the range [-1, 1]
max_X = np.max(X_train)
factor = max_X / 2
X_train = (X_train - factor) / factor

max_Y = np.max(Y_train)
factor = max_X / 2
Y_train = (Y_train - factor) / factor

In [ ]:
## Original
generator = Sequential([
        Dense((subSquareSize[0]/4)*(subSquareSize[1]/4)*16, 
              input_dim=subSquareSize[0]*subSquareSize[1], 
              activation=LeakyReLU(0.2)),
        BatchNormalization(),
        Reshape((subSquareSize[0]/4,subSquareSize[0]/4,16)),
        UpSampling2D(),
        Convolution2D(8, (5, 5), padding='same', activation=LeakyReLU(0.2)),
        BatchNormalization(),
        UpSampling2D(),
        Convolution2D(1, (5, 5), padding='same', activation='tanh')
    ])
generator.summary()

In [ ]:
# One to use for waterways
generator = Sequential([
        #BatchNormalization(),
        Convolution2D(4, (5, 5), input_shape=(subSquareSize[0],subSquareSize[1],1), padding='same', activation=LeakyReLU(0.2)),
        BatchNormalization(),
        Reshape((subSquareSize[0]*subSquareSize[1]*4,), input_shape=(subSquareSize[0],subSquareSize[1],4)),
        Dense((subSquareSize[0]*subSquareSize[1]*4), input_dim=(subSquareSize[0]*subSquareSize[1]*4), activation=LeakyReLU(0.2)),
        Reshape((subSquareSize[0],subSquareSize[1],4), input_shape=(subSquareSize[0]*subSquareSize[1]*4,)),
        #UpSampling2D(),
        Convolution2D(4, (5, 5), padding='same', activation=LeakyReLU(0.2)),
        BatchNormalization(),
        #UpSampling2D(),
        Convolution2D(1, (5, 5), padding='same', activation='tanh'),
        #Concatenate()
    
    ])
generator.summary()

In [ ]:
discriminator = Sequential([
        Convolution2D(64, 5, 5, subsample=(2,2), input_shape=(subSquareSize[0]*2,subSquareSize[1],1), border_mode='same', activation=LeakyReLU(0.2)),
        Dropout(0.3),
        Convolution2D(128, 5, 5, subsample=(2,2), border_mode='same', activation=LeakyReLU(0.2)),
        Dropout(0.3),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
discriminator.summary()

In [ ]:
generator.compile(loss='binary_crossentropy', optimizer=Adam())

In [ ]:
discriminator.compile(loss='binary_crossentropy', optimizer=Adam())

In [ ]:
discriminator.trainable = False
ganInput = Input(shape=(subSquareSize[0], subSquareSize[1],1))
# getting the output of the generator
# and then feeding it to the discriminator
# new model = D(G(input))
x = generator(ganInput)
#catLayer = Sequential()
#discInput = Concatenate([x,ganInput], input_shape=(subSquareSize[0]*2, subSquareSize[1]))
#catLayer.add(Concatenate([ganInput,ganInput,ganInput], input_shape=(subSquareSize[0]*2, subSquareSize[1])))
#catLayer.add(keras.layers.concatenate([x,ganInput], input_shape=(subSquareSize[0]*2, subSquareSize[1])))
discInput = keras.layers.concatenate([x,ganInput], 
                                     axis=1,
                                     input_shape=(subSquareSize[0]*2, subSquareSize[1]))
#discInput = Input([x,ganInput])
ganOutput = discriminator(discInput)
#ganOutput = discriminator(x)

gan = Model(input=ganInput, output=ganOutput)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

In [ ]:
gan.summary()

In [ ]:
def train(epoch=10, batch_size=128):
    batch_count = X_train.shape[0] // batch_size
    
    for i in range(epoch):
        for j in tqdm(range(batch_count)):
            # Input for the generator
            #noise_input = np.random.rand(batch_size, 100)
            
            # getting random images from X_train of size=batch_size 
            # these are the real images that will be fed to the discriminator
            image_batch_indices = np.random.randint(0, X_train.shape[0], size=batch_size)
            mask_image_batch = np.zeros((batch_size, subSquareSize[0],subSquareSize[1],1))
            for i in range(batch_size):
                mask_image_batch[i,:,:,0] = X_train[image_batch_indices[i],:,:]#.flatten()
            #mask_image_batch = X_train[image_batch_indices]
            #print('mask_image_batch.shape', mask_image_batch.shape)
            real_image_batch = np.zeros((batch_size, subSquareSize[0], subSquareSize[1], 1))
            real_image_batch[:,:,:,0] = Y_train[image_batch_indices]
            
            # these are the predicted images from the generator
            predictions = generator.predict(mask_image_batch, batch_size=batch_size)#[:,:,:,0]
            predictionsAndMasks = np.concatenate([predictions, mask_image_batch], axis=1)
            realImagesAndMasks = np.concatenate([real_image_batch, mask_image_batch], axis=1)
            # the discriminator takes in the real images and the generated images
            #print(predictions.shape, real_image_batch.shape)
            #X = np.concatenate([predictions, real_image_batch], axis=0)
            #print('X.shape', X.shape)
            X = np.concatenate([predictionsAndMasks, realImagesAndMasks], axis=0)
            #X = np.concatenate([predictions, real_image_batch], axis=0)
            
            # labels for the discriminator
            y_discriminator = [0]*batch_size + [1]*batch_size
            
            # Let's train the discriminator
            discriminator.trainable = True
            loss = discriminator.train_on_batch(X, y_discriminator)
            print('epoch %i loss %f' %(epoch, loss))
            # Let's train the generator
            #noise_input = np.random.rand(batch_size, 100)
            image_batch_indices = np.random.randint(0, X_train.shape[0], size=batch_size)
            mask_image_batch_2 = np.zeros((batch_size, subSquareSize[0],subSquareSize[1],1))
            for i in range(batch_size):
                mask_image_batch_2[i,:,:,0] = X_train[image_batch_indices[i],:,:]#.flatten()
            discriminator.trainable = False
            y_generator = [1]*batch_size
            gan.train_on_batch(mask_image_batch_2, y_generator)

In [ ]:
train(30, 128)

In [ ]:
def plot_output():
    nToPlot = 25
    #try_input = (np.random.rand(100,subSquareSize[0]*subSquareSize[1]))
    #try_input = np.zeros((100,100)) + 1
    #try_input[30:70,:] += 1.5
    #try_input[70:100,:] += 2.5
    #try_input = np.array(try_input)
    image_batch_indices = np.random.randint(0, X_train.shape[0], size=nToPlot)
    mask_image_batch = np.zeros((nToPlot, subSquareSize[0],subSquareSize[1],1))
    for i in range(nToPlot):
        mask_image_batch[i,:,:,0] = X_train[image_batch_indices[i],:,:]#.flatten()
    preds = generator.predict(mask_image_batch)
    predsAndMasks = np.concatenate([mask_image_batch, preds], axis=1)
    #preds = generator.predict(np.array([[1]*10,2,2,2,3,3,3]))
    plt.clf()
    plt.figure(figsize=(10,10))
    for i in range(preds.shape[0]):
        plt.subplot(5, 5, i+1)
        #plt.imshow(preds[i, :, :, 0], cmap='gray')
        plt.imshow(predsAndMasks[i, :, :, 0], cmap='gray')
        plt.axis('off')
    
    # tight_layout minimizes the overlap between 2 sub-plots
    plt.tight_layout()

In [ ]:
plot_output()

In [ ]:
print('Saving 30')
generator.save_weights('2018_06_04_waterways_gen_30.h5')
discriminator.save_weights('2018_06_04_waterways_dis_30.h5')
#plot_output()
train(30, 128)
print('Saving 60')
generator.save_weights('2018_06_04_waterways_gen_60.h5')
discriminator.save_weights('2018_06_04_waterways_dis_60.h5')
#plot_output()
train(30, 128)
print('Saving 90')
generator.save_weights('2018_06_04_waterways_gen_90.h5')
discriminator.save_weights('2018_06_04_waterways_dis_90.h5')
train(30, 128)
#plot_output()
print('Saving 120')
generator.save_weights('2018_06_04_waterways_gen_120.h5')
discriminator.save_weights('2018_06_04_waterways_dis_120.h5')
train(30, 128)
#plot_output()
print('Saving 150')
generator.save_weights('2018_06_04_waterways_gen_150.h5')
discriminator.save_weights('2018_06_04_waterways_dis_150.h5')
#plot_output()

In [ ]:
batch_size = 10
# getting random images from X_train of size=batch_size 
# these are the real images that will be fed to the discriminator
image_batch_indices = np.random.randint(0, X_train.shape[0], size=batch_size)
mask_image_batch = np.zeros((batch_size, subSquareSize[0],subSquareSize[1],1))
for i in range(batch_size):
    mask_image_batch[i,:,:,0] = X_train[image_batch_indices[i],:,:]#.flatten()
#mask_image_batch = X_train[image_batch_indices]
#print('mask_image_batch.shape', mask_image_batch.shape)
real_image_batch = np.zeros((batch_size, subSquareSize[0], subSquareSize[1], 1))
real_image_batch[:,:,:,0] = Y_train[image_batch_indices]
            
# these are the predicted images from the generator
predictions = generator.predict(mask_image_batch, batch_size=batch_size)#[:,:,:,0]
predictionsAndMasks = np.concatenate([predictions, mask_image_batch], axis=1)
realImagesAndMasks = np.concatenate([real_image_batch, mask_image_batch], axis=1)
# the discriminator takes in the real images and the generated images
#print(predictions.shape, real_image_batch.shape)
#X = np.concatenate([predictions, real_image_batch], axis=0)
#print('X.shape', X.shape)
X = np.concatenate([predictionsAndMasks, realImagesAndMasks], axis=2)
#X = np.concatenate([predictions, real_image_batch], axis=0)
            
# labels for the discriminator
y_discriminator = [0]*batch_size + [1]*batch_size
            
# Let's train the discriminator
discriminator.trainable = True
result = discriminator.train_on_batch(X, y_discriminator)


# See why this doesn't look so great

In [ ]:
print(discriminator.metrics_names)
print(result)
plt.clf()
plt.figure(figsize=(10,10))
for i in range(X.shape[0]):
    plt.subplot(5, 5, i+1)
    #plt.imshow(preds[i, :, :, 0], cmap='gray')
    #plt.imshow(predictionsAndMasks[i, :, :, 0], cmap='gray')
    plt.imshow(X[i, :, :, 0], cmap='gray')
    plt.axis('off')
    
# tight_layout minimizes the overlap between 2 sub-plots
plt.tight_layout()

# I think this means the discriminator is too good